In [1]:
!mkdir -p data
!mkdir -p data_input

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
df_enc = pd.read_csv('data_win/enc.csv').set_index('hosp_id')
df_pop = pd.read_csv('data_win/windows.csv').set_index('hosp_id')
df_win = pd.read_csv('data_win/windows_map.csv')

## Filter vars

### Flow

In [4]:
df_meds = pd.read_pickle('data_win/flow-2020_0701-win.p')

In [5]:
len(df_meds)

91617339

In [6]:
# Filter by year
df_meds = df_meds[df_meds['hosp_id'].isin(df_enc.index)]

In [ ]:
df_meds

In [8]:
keep = json.load(open('../NEW-flow/_select_flow/flow_part1.json', 'r'))

In [9]:
df_out = df_meds[df_meds['observationtermid'].isin(keep)]

In [ ]:
df_out

In [11]:
df_out.to_pickle('data/flow-filtered-20.p')

## Format IDs

### Meds 20

In [12]:
df_meds = pd.read_pickle('data/flow-filtered-20.p')

In [13]:
df_meds['observationtermid'].nunique()

595

In [14]:
len(df_meds)

45108578

In [15]:
df_meds_small = df_meds[df_meds['hosp_id'].isin(df_enc.index)]

In [16]:
len(df_meds_small)

45108578

In [17]:
df_meds_small = df_meds_small[(df_meds_small['t'] >= 0)]

In [18]:
len(df_meds_small)

45108578

In [19]:
df_meds_ID = df_meds.set_index(['hosp_id', 'window_id']).join(df_win.set_index(['hosp_id', 'window_id'])[['ID']]).reset_index()

In [20]:
df_meds_ID.to_pickle('data_input/flow_ID-20.p')

In [21]:
df_attr = df_meds_ID[['ID', 'window_t', 'observationtermid', 'result']] \
.rename(columns={'window_t': 't', 'observationtermid': 'variable_name', 'result': 'variable_value'})
df_attr = df_attr.drop_duplicates(subset=['ID', 't', 'variable_name'], keep='last') # Need to improve handling of duplicates

In [22]:
df_attr['variable_name'] = df_attr['variable_name'].apply(lambda s: '_' + str(s))

In [ ]:
df_attr

In [24]:
df_attr.to_pickle('data_input/flow-20.p')